# LLMFS Load gpt2 weights into pytorch model
Extracts from LLMFS to load weights into pytorch implementation of GPT2. Test different layers with simple values to compare with Tensorflow model.

In [1]:
!pip -q install tiktoken

In [8]:
from importlib.metadata import version

pkgs = ["matplotlib", 
        "numpy", 
        "tiktoken", 
        "torch",
        "tensorflow" # For OpenAI's pretrained weights
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.7.3
numpy version: 1.26.3
tiktoken version: 0.12.0
torch version: 2.1.1+cu121
tensorflow version: 2.15.0


In [59]:
import numpy as np
np.set_printoptions(precision=5)
def tldr(tensor):
    return f"{tensor.dtype} {tensor.shape} {tensor[0][0][:3]}"

### 5.1.1 Using GPT to generate text

In [3]:
import torch
from previous_chapters import GPTModel
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

In [4]:
import tiktoken
from previous_chapters import generate_text_simple

# Alternatively:
# from llms_from_scratch.ch04 import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


### 5.3.3 Modifying the text generation function

In [5]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for i in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        print("i=", i, "idx_cond=", idx_cond)
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        # print("  logits=", logits.shape)

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break
        # print("  idx_next=", idx_next)
        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

In [6]:
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

i= 0 idx_cond= tensor([[6109, 3626, 6100,  345]])
i= 1 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240]])
i= 2 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686]])
i= 3 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611]])
i= 4 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876]])
i= 5 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876, 48215]])
i= 6 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876, 48215, 21196]])
i= 7 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876, 48215, 21196,
           994]])
i= 8 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876, 48215, 21196,
           994, 19513]])
i= 9 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876, 48215, 21196,
           994, 19513, 31728]])
i= 10 idx_cond= tensor([[ 6109,  3626,  6100,   345, 41240, 11686, 28611, 38876, 48215, 21196,
           994, 19513, 3172

## 5.5 Loading pretrained weights from OpenAI

In [9]:
print("TensorFlow version:", version("tensorflow"))
print("tqdm version:", version("tqdm"))

TensorFlow version: 2.15.0
tqdm version: 4.66.1


In [10]:
from gpt_download import download_and_load_gpt2

2025-11-27 00:04:50.319964: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-27 00:04:50.799635: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-27 00:04:50.799733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-27 00:04:50.908184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-27 00:04:51.141567: I tensorflow/core/platform/cpu_feature_guar

In [11]:
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

File already exists and is up-to-date: gpt2/124M/checkpoint
File already exists and is up-to-date: gpt2/124M/encoder.json
File already exists and is up-to-date: gpt2/124M/hparams.json
File already exists and is up-to-date: gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/124M/model.ckpt.index
File already exists and is up-to-date: gpt2/124M/model.ckpt.meta
File already exists and is up-to-date: gpt2/124M/vocab.bpe


In [12]:
print("Settings:", settings)

Settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}


In [13]:
print("Parameter dictionary keys:", params.keys())

Parameter dictionary keys: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


In [60]:
print(params["wte"])
print("Token embedding weight tensor dimensions:", params["wte"].shape)

[[-0.1101  -0.03927  0.03311 ... -0.13637  0.01506  0.04532]
 [ 0.04034 -0.04862  0.04625 ...  0.08605  0.00254  0.04319]
 [-0.12746  0.04794  0.1841  ...  0.08992 -0.12972 -0.08786]
 ...
 [-0.04454 -0.05484  0.01226 ...  0.10435  0.09783 -0.06953]
 [ 0.18601  0.01666  0.04612 ... -0.09625  0.07848 -0.02246]
 [ 0.05135 -0.02769  0.04994 ...  0.00705  0.1552   0.12068]]
Token embedding weight tensor dimensions: (50257, 768)


In [61]:
# Define model configurations in a dictionary for compactness
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # Example model name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [62]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [63]:
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])
    
    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight, 
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias, 
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight, 
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias, 
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight, 
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias, 
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale, 
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift, 
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale, 
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift, 
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])
    

In [64]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_weights_into_gpt(gpt, params)
gpt.to(device);

In [65]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


i= 0 idx_cond= tensor([[6109, 3626, 6100,  345]], device='cuda:0')
i= 1 idx_cond= tensor([[6109, 3626, 6100,  345,  355]], device='cuda:0')
i= 2 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290]], device='cuda:0')
i= 3 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355]], device='cuda:0')
i= 4 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355,  262]], device='cuda:0')
i= 5 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355,  262, 1021]],
       device='cuda:0')
i= 6 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355,  262, 1021,  460]],
       device='cuda:0')
i= 7 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355,  262, 1021,  460,  467]],
       device='cuda:0')
i= 8 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355,  262, 1021,  460,  467, 1566]],
       device='cuda:0')
i= 9 idx_cond= tensor([[6109, 3626, 6100,  345,  355, 1290,  355,  262, 1021,  460,  467, 1566,
          262]], device='cuda:0')
i= 10 idx_cond= tensor(

In [66]:
idx=text_to_token_ids("Every effort moves you", tokenizer)
idx.shape

torch.Size([1, 4])

In [67]:
idx

tensor([[6109, 3626, 6100,  345]])

### Test embedding layer

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)
gpt.to("cpu");
x_effort = torch.tensor([[6109, 3626, 6100, 345]])
batch_size, seq_len = x_effort.shape

tok_emb = gpt.tok_emb
pos_emb = gpt.pos_emb
tok_embeds = tok_emb(x_effort)
pos_embeds = pos_emb(torch.arange(seq_len, device=x.device))
x_effort_emb = tok_embeds + pos_embeds

print("x_effort_emb:", tldr(x_effort_emb))
assert list(x_effort_emb.shape) == [1, 4, 768]
assert np.allclose(x_effort_emb.detach()[0][0][:3], np.array([0.0793, -0.2979 ,  0.0882]), rtol=1e-4, atol=1e-4)

using device: cuda
x_effort_emb: torch.float32 torch.Size([1, 4, 768]) tensor([ 0.0793, -0.2979,  0.0882], grad_fn=<SliceBackward0>)


### Test norm1

In [69]:
# dir(gpt.trf_blocks)
# gpt.trf_blocks
tb0_norm1 = gpt.trf_blocks.get_submodule("0.norm1")
# dir(tb0_norm1)
x = torch.tensor(np.ones((1, 768) , dtype=np.float32))
# tb0_norm1(x) # [-3.6773e-03,  2.7197e-02, -6.4041e-02

# tb11_norm1 = gpt.trf_blocks.get_submodule("11.norm1")
#tb11_norm1(x) # [ 5.0957e-02,  5.3063e-03,  7.1952e-02

In [72]:
gpt.trf_blocks.get_submodule("0.norm1")(x_effort_emb)

tensor([[[ 0.0472, -0.1166, -0.0254,  ..., -0.0134, -0.0348,  0.0272],
         [ 0.0059, -0.1736, -0.0962,  ...,  0.1672,  0.0370, -0.0667],
         [-0.0523, -0.0529,  0.1247,  ...,  0.0327,  0.1120,  0.0014],
         [-0.0406,  0.0055,  0.0506,  ..., -0.1240, -0.0731, -0.0745]]],
       grad_fn=<AddBackward0>)

In [77]:
# gpt.trf_blocks.get_submodule("0.norm1").shift

In [24]:
# test query layer, key layer, value layer
x = torch.tensor(np.ones((1, 768) , dtype=np.float32))

# block 0
att0 = gpt.trf_blocks.get_submodule("0.att")
query0 = att0.get_submodule("W_query")
# query0(x) # [-1.3708e+01,  1.3385e+01,  1.4323e+01
key0 = att0.get_submodule("W_key")
# key0(x) # [ 1.8049e-01, -1.4381e-01,  6.2964e-01
value0 = att0.get_submodule("W_value")
# value0(x)
proj0 = att0.get_submodule("out_proj")
# proj0(x)
# proj0(value0(key0(query0(x)))) # [-2.3273e+01, -7.9272e+02,  5.6245e+02

# block 11
att11 = gpt.trf_blocks.get_submodule("11.att")
query11 = att11.get_submodule("W_query")
# query11(x) # [-5.4209e+00,  4.6236e+00,  4.5401e+00
key11 = att11.get_submodule("W_key")
# key11(x) # [ 5.8911e+00, -3.3184e-01,  6.3656e-01
value11 = att11.get_submodule("W_value")
# value11(x) # [-1.2480e+00, -3.0783e+00,  5.9679e+00

proj11 = att11.get_submodule("out_proj")
# proj11(x) # [-4.1535e-01,  2.1763e+00,  4.7958e-01

# proj11(value11(key11(query11(x)))) # [ 3.4414e+02,  4.9568e+02,  3.8639e+02


In [25]:

x = torch.tensor(np.ones((1, 768) , dtype=np.float32))
tb0_norm2 = gpt.trf_blocks.get_submodule("0.norm2")
# tb0_norm2(x)

tb0_ff = gpt.trf_blocks.get_submodule("0.ff")
tb0_ff = gpt.trf_blocks.get_submodule("0.ff")
#tb0_ff(x) # [-1.6735e+01, -6.9883e+00,  4.1138e+00

# tb11_norm2 = gpt.trf_blocks.get_submodule("11.norm2")
# tb11_norm2(x)
tb11_ff = gpt.trf_blocks.get_submodule("11.ff")
#tb11_ff(x) # [ 1.3675e+01,  2.2839e+01, -1.7306e+01

In [26]:
tb0_ff_linear1 = tb0_ff.layers[0]
tb0_ff_gelu = tb0_ff.layers[1]
# These 2 layers composed correspond to linear perceptron_layer with gelu activation function
# tb0_ff_gelu(tb0_ff_linear1(x)) # [ 3.5592, -0.1381, -0.1655

In [27]:
tb0_ff_linear2 = tb0_ff.layers[2]
# tb0_ff_linear2(x) # How to test??

In [28]:
x = torch.tensor(np.ones((1, 768) , dtype=np.float32))
final_norm = gpt.final_norm
# final_norm(x) # [ 1.0872e-03,  3.6529e-02, -6.7296e-02


In [29]:
x = torch.tensor(np.ones((1, 768) , dtype=np.float32))
out_head = gpt.out_head
out_head
#out_head(x) # [ 0.3766,  3.4404,  2.0287

Linear(in_features=768, out_features=50257, bias=False)

In [30]:
x_trivial = torch.tensor([[1, 2, 3]])
gpt(x_trivial)

tensor([[[-32.9011, -31.2024, -34.6623,  ..., -39.4868, -39.8732, -32.2387],
         [-55.5208, -53.4286, -56.4767,  ..., -68.1539, -66.7709, -58.6006],
         [-61.7969, -60.5386, -59.5503,  ..., -75.3206, -72.7731, -65.5706]]],
       grad_fn=<UnsafeViewBackward0>)

In [31]:
context_size=GPT_CONFIG_124M["context_length"]
idx = text_to_token_ids("Every effort moves you", tokenizer)

# x # tensor([[6109, 3626, 6100,  345]])
idx_cond = idx[:, -context_size:]
idx_cond # tensor([[6109, 3626, 6100,  345]])
logits = gpt(idx)
print("logits:\n", logits)
logits = logits[:, -1, :]
print("logits:\n", logits)
print("logits[-1]:\n", logits[-1])
probas = torch.softmax(logits, dim=-1)
print("probas:\n", probas)
idx_next = torch.argmax(probas, dim=-1)
print("idx_next:\n", idx_next)

logits:
 tensor([[[ -35.5820,  -34.9804,  -38.4522,  ...,  -42.0959,  -41.8533,
           -35.5966],
         [ -76.9601,  -76.6970,  -81.9309,  ...,  -88.7984,  -86.7631,
           -78.9627],
         [-125.3487, -126.2704, -135.0948,  ..., -132.3173, -135.2544,
          -127.6511],
         [-136.6002, -137.3804, -146.5556,  ..., -148.2978, -147.2155,
          -139.5677]]], grad_fn=<UnsafeViewBackward0>)
logits:
 tensor([[-136.6002, -137.3804, -146.5556,  ..., -148.2978, -147.2155,
         -139.5677]], grad_fn=<SliceBackward0>)
logits[-1]:
 tensor([-136.6002, -137.3804, -146.5556,  ..., -148.2978, -147.2155,
        -139.5677], grad_fn=<SelectBackward0>)
probas:
 tensor([[1.6013e-03, 7.3391e-04, 7.6013e-08,  ..., 1.3313e-08, 3.9292e-08,
         8.2357e-05]], grad_fn=<SoftmaxBackward0>)
idx_next:
 tensor([2651])


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpt.to(device);
token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=5,
    context_size=NEW_CONFIG["context_length"],
)
print(token_ids)

i= 0 idx_cond= tensor([[6109, 3626, 6100,  345]], device='cuda:0')
i= 1 idx_cond= tensor([[6109, 3626, 6100,  345, 2651]], device='cuda:0')
i= 2 idx_cond= tensor([[6109, 3626, 6100,  345, 2651,   13]], device='cuda:0')
i= 3 idx_cond= tensor([[6109, 3626, 6100,  345, 2651,   13,  198]], device='cuda:0')
i= 4 idx_cond= tensor([[6109, 3626, 6100,  345, 2651,   13,  198,  198]], device='cuda:0')
tensor([[6109, 3626, 6100,  345, 2651,   13,  198,  198,  464]],
       device='cuda:0')


In [33]:
gpt.to("cpu")
start_context = "Every effort moves you"
token_ids = generate_text_simple(
    model=gpt,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=256
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you forward.

The first step is to understand


In [34]:
#
#
#

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
gpt.to("cpu");
# x = torch.tensor([[1]])
x_trivial = torch.tensor([[1, 2, 3]])

tok_embeds = tok_emb(x_trivial)
pos_embeds = pos_emb(torch.arange(seq_len, device=x.device))
x_embedded = tok_embeds + pos_embeds
x_embedded # [ 2.1520e-02, -2.4603e-01,  5.0275e-02
att0 = gpt.trf_blocks.get_submodule("0.att")
query0 = att0.get_submodule("W_query")
query0(x_embedded)

cuda


tensor([[[-1.4599,  2.6167,  3.0796,  ..., -3.7902, -0.3126, -3.3074],
         [-0.9835,  2.6840,  1.8440,  ..., -0.2896, -3.1945, -1.4446],
         [-0.1488,  2.3184,  1.7861,  ..., -1.1154, -3.3635, -2.2503]]],
       grad_fn=<ViewBackward0>)

In [41]:
norm10 = gpt.trf_blocks.get_submodule("0.norm1")

In [44]:
x01 = norm10(x_embedded)
print('x01:', x01[0][0][:3].detach().numpy())


x01: [ 0.00984593 -0.09290446 -0.04286208]


In [45]:
att0 = gpt.trf_blocks.get_submodule("0.att")
x02 = att0(x01)
print('x02:', x02[0][0][:3].detach().numpy())

x02: [0.3328247  0.08782893 0.18114716]
